In [1]:
# This script requires the following for each participant
# 1) Classifier output from the structural MRI analysis - this should be a probability map between 0 and 1. 
#(see FCDdetection scripts for how to run a neural network to detect FCDs)
# 2) A .fcsv file for each electrode implanted. This should contain the co-ordinates of each contact on the electrode
# 3) A spreadsheet of participants, electrodes implanted, ictal contacts (see example on github)

#This script does the following:
#1. Load in the classifier output from the classifier training cohort and find the optimal threshold to threshold the probability map
#2. Calculates the sensitivity of the classifier on the training cohort at this threshold
#3. Load in the classifier output tested on the controls and finds out the specificity at this optimal threshold
#4. Load in the SEEG patients' classifier output, threshold and cluster their outputs 
#5. Save an interactive plot as a .html file of the electrodes and the clusters in the SEEG patients
#6. Plot the relationship of the ictal electrodes to the top cluster




In [ ]:
# Import necessary packages
import io_meld as io
import numpy as np
import os
import mesh_tools as mt
from matplotlib import pyplot as plt
import glob
%matplotlib inline
import subprocess


import pandas as pd
from nilearn import plotting, surface,datasets
%matplotlib notebook
from matplotlib import cm as mpl_cm
from nilearn.plotting import cm
import matplotlib as mpl
from matplotlib.colors import Normalize, LinearSegmentedColormap
import io_mesh
import json
from scipy import sparse
from scipy import ndimage
from nilearn import datasets
import nilearn
import numbers
from nilearn.plotting.js_plotting_utils import (add_js_lib, HTMLDocument, mesh_to_plotly,
                                encode, colorscale, get_html_template,
                                to_color_strings)
import argparse

from nilearn.plotting.js_plotting_utils import (add_js_lib, HTMLDocument, mesh_to_plotly,
                                encode, colorscale, get_html_template,
                                to_color_strings)
from nilearn.surface import load_surf_data, load_surf_mesh, vol_to_surf

from nilearn._utils.compat import _basestring
import PtitPrince as pt
import seaborn as sns 

In [ ]:
#Define functions used
def get_neighbours_from_tris(tris, label=None):
    """Get surface neighbours from tris
        Input: tris
         Returns Nested list. Each list corresponds 
        to the ordered neighbours for the given vertex"""
    n_vert=np.max(tris)+1
    neighbours=[[] for i in range(n_vert)]
    for tri in tris:
        neighbours[tri[0]].extend([tri[1],tri[2]])
        neighbours[tri[2]].extend([tri[0],tri[1]])
        neighbours[tri[1]].extend([tri[2],tri[0]])
    #Get unique neighbours
    for k in range(len(neighbours)):      
        if label is not None:
            neighbours[k] = set(neighbours[k]).intersection(label)
        else :
            neighbours[k]=f7(neighbours[k])
    return neighbours;


def f7(seq):
    """returns uniques but in order of original appearance.
    Used to retain neighbour triangle relationship"""
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))];

def flatten(nested_list):
        """flatten nested list"""
        #test if nested
        if any(isinstance(i, list) for i in nested_list):
            flat_list = [item for sublist in nested_list for item in sublist]
            return flat_list
        else:
            return nested_list

In [ ]:
#Define functions used
def cluster_above_threshold(overlay, neighbours,threshold):
    """cluster outputs above threshold and returns cluster index at vertex locations"""
    unranked=overlay>=threshold
    clust_ind=0
    clusters=np.zeros(len(overlay))
    while np.sum(unranked)>0:
        cluster=np.zeros(len(overlay))
        clust_ind+=1
        new_verts=np.argmax(unranked)
        unranked[new_verts]=0
        cluster[new_verts]=clust_ind
        old_len=np.sum(unranked)+1
        while np.sum(unranked)<old_len:
            neighbours_vert=np.zeros(len(overlay))
            try :
                neighbours_vert[flatten(neighbours[new_verts])]=1
            except TypeError:
                neighbours_vert[neighbours[new_verts]]=1
            new_verts=np.logical_and(neighbours_vert,unranked)
            old_len=np.sum(unranked)
            cluster[new_verts]=clust_ind
            unranked[new_verts]=0
        clusters+=cluster
    return clusters.astype(int)

def filter_cluster(clusters,areas, minimum_area):
    """filter out clusters smaller than minimum areas"""
    #get existing clusters
    clusters_indices=np.unique(clusters)
    #start after the zero index
    filtered_clusters=clusters.copy()
    #store which vertices grew
    for c in clusters_indices[1:]:
        #calculate cluster area
        area=np.sum(areas[clusters==c])
        #if cluster area below threshold, set all vertices to zero
        if area < minimum_area:
            filtered_clusters[clusters==c]=0
    return filtered_clusters

def clump_clusters(clusters,neighbours, n_steps=2):
    """group clusters separated by n_steps"""
    cluster_indices=np.unique(clusters)
    grouped_clusters=clusters.copy()
    growth_rings=np.zeros((len(cluster_indices),len(clusters)))
    for c in cluster_indices[1:]:
        cluster=clusters==c
        #grow out cluster
        for step in range(n_steps):
            cluster=np.unique(flatten(neighbours[cluster]))
        #check if grown cluster contains new indices from nearby clusters
        indices_in_grown_cluster=np.unique(clusters[cluster])
        if len(indices_in_grown_cluster)>2:
            #set those clusters to that value
            growth_rings[c,cluster]=1
            for index in indices_in_grown_cluster[1:]:
                #look up in grouped clusters to pass forward any changed values from previous iterations
                if grouped_clusters[clusters==c][0] >0:
                    new_value = grouped_clusters[clusters==c][0]
                    sum_growth=growth_rings[new_value,:]+growth_rings[c,:]
                    link=sum_growth==2
                    #print(link)
                else:
                    new_value= c
                grouped_clusters[clusters==index]=new_value
               # grouped_clusters[link]=new_value
    return grouped_clusters
            #save growth ring
            

from scipy.stats import skew
def rank_clusters(clusters,overlay,ranking="mean"):
    """reorder indexing to order cluster by mean/max value """
    clusters_indices=np.unique(clusters)[1:]
    values=np.zeros(len(clusters_indices))
    for k,c in enumerate(clusters_indices):
        #calculate cluster area
        if ranking=="mean":
            values[k]=np.mean(overlay[clusters==c])
        elif ranking=="max":
            values[k]=np.max(overlay[clusters==c])
        elif ranking=="skew":
            values[k]=skew(overlay[clusters==c])
    ranked_clusters=clusters.copy()
    order=np.argsort(values)
    for k,i in enumerate(order):
        ranked_clusters[clusters==clusters_indices[k]]=i+1
    return ranked_clusters

def check_cluster_overlap_with_lesion_mask(clusters, mask, cluster_index=1):
    """test if lesion mask overlaps with a specific cluster
    cluster_index - default tests just 1 top cluster overlap"""
    return np.logical_and(clusters==cluster_index,mask==1).any()

def check_any_overlap_with_lesion_mask(clusters, mask):
    """test if lesion mask overlaps with any of the clusters"""
    return np.logical_and(clusters>0,mask==1).any()


def return_cluster_overlap_with_lesion_mask(clusters, mask):
    """test if lesion mask overlaps with any of the clusters
    cluster_index - default tests just 1 top cluster overlap"""
    overlaps=np.unique(clusters[np.logical_and(clusters>0,mask==1)])
    if overlaps.any():
        return overlaps
    else: 
        return [0]
    
def get_optimal_threshold(sensitivity, specificity, thresholds, resolution=10):
    """calculate optimal sensitivity"""
    y=sensitivity+specificity
    x=thresholds
    upsampled=np.repeat(y,10)
    smoothed=ndimage.filters.gaussian_filter1d(upsampled,resolution)
    upsampled_thresholds=np.linspace(0,1,len(thresholds)*resolution)
    return np.round(upsampled_thresholds[np.argmax(smoothed)],2)


def defrag_surface(clusters,neighbours,steps=5):
    """ Removes small holes in clusters by expanding and shrinking the cluster"""
    #find basic lesion vertics
    cluster_indices=np.unique(clusters)[1:]
    clusters_defragged=clusters.copy()
    for cluster in cluster_indices:
        patch=clusters==cluster
        #expanding stage. add neighbours
        for k in range(steps):
            patch[flatten(neighbours[patch])]=True
        #shrinking stage. remove edge vertices
        not_patch=patch==False
        for k in range(steps):
            not_patch[flatten(neighbours[not_patch])]=True
        clusters_defragged[not_patch==False]=cluster
    return clusters_defragged;



In [2]:
def label2object(outname,grey,white,label,shrink=False):
    """reduce object to just the label vertices
    NB if there are isolated, unconnected vertices this causes bugs
    shrink - removes non-label vertices"""
    gsurf=io_mesh.load_mesh_geometry(grey)
    wsurf=io_mesh.load_mesh_geometry(white)
    label_ones=np.zeros(len(gsurf['coords']))
    label_ones[label]=1
    #extract only triangles inside label
    labeltris=[]
    for tri in gsurf['faces']:
        if sum(label_ones[tri]) == 3:
            labeltris.append(tri.tolist())
    label_surf={}
    if shrink:
        labeltris=flatten(labeltris)
        newlabeltris=[]
        #create new indexing
        labellist=sorted(label.tolist())
        for l in labeltris:
            newlabeltris.append(labellist.index(l))
        newlabeltris=np.reshape(newlabeltris,(len(newlabeltris)/3,3))
        #create new object
        label_surf['coords']=np.vstack((gsurf['coords'][label],wsurf['coords'][label]))
        label_surf['faces']=np.vstack((np.array(newlabeltris),np.array(newlabeltris)+len(label)))
    else:
        label_surf['coords']=np.vstack((gsurf['coords'],wsurf['coords']))
        label_surf['faces']=np.vstack((np.array(labeltris),np.array(labeltris)+len(gsurf['coords'])))
    io_mesh.save_mesh_geometry(outname,label_surf)
    return

def tidy_integer_labels(label,neighbours):
    """ tidy overlay intended to be integers, but with interpolation errors
    find all values greater than zero. if connected to higher value, set as higher value"""
    vertices=np.where(label>0)[0]
    change=1
    #count number fo vertices that have changed, keep iterating until change stops
    while change>0:
        #store the sum of label values from previous iteration
        old_sum=np.sum(label)
        #for each non-zero vertex...
        for vertex in vertices:
            #find the highest value from among the neighbours
            max_n=np.max(label[neighbours[vertex]])
            #if the highest neighbouring value is above the value of the vertex, replace with neighbour value
            if max_n>=label[vertex] :
                label[vertex]=max_n
            #otherwise set it to zero
            else:
                label[vertex]=0
        change=np.sum(label)-old_sum
        vertices=np.where(label>0)[0]
    non_ints=np.where(label%1>0)[0]
    label[non_ints]=0
    return label.astype(int)
        
    
def lesion_coordinates(grey,white,label_name,cluster_no=1):
    """returns coordinates of grey and white surfaces of lesion defined by label"""
    gsurf=io_mesh.load_mesh_geometry(grey)
    wsurf=io_mesh.load_mesh_geometry(white)
    neighbours=get_neighbours_from_tris(gsurf['faces'])
    label=tidy_integer_labels(np.array(io.import_mgh(label_name)),neighbours)    
    lesion_coords=np.vstack((gsurf['coords'][label==cluster_no],wsurf['coords'][label==cluster_no]))
    return lesion_coords

# Define functions for seeg data
def get_fiducials(subject_id):
    """get co-ordinates of electrode contacts 'fiducials' from a dataframe """
    fiducials=np.array(Fiducials[subject_id]).astype(str)
    fiducials = fiducials[fiducials != 'nan']
    fiducials = fiducials[fiducials != 'NaN']
    return fiducials

def get_electrodes(sheet, subject_id):
    """ Retrieves the list of electrodes from the spreadsheet"""
    row = np.array(sheet.loc[sheet['ID'] == subject_id]).astype(str)[0][1:]
    row = row[row!='nan']
    return row

def closest_node(node, nodes):
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argmin(dist_2), np.sqrt(np.min(dist_2))

def _get_markers(coords, colors):
    """ create connectome dictionary containing markers (electrode coordinates)
    Colour each marker by colors, an array specifiying the desired colours.  """
    connectome = {}
    coords = np.asarray(coords, dtype='<f4')
    x, y, z = coords.T
    for coord, cname in [(x, "x"), (y, "y"), (z, "z")]:
        connectome["_con_{}".format(cname)] = encode(
            np.asarray(coord, dtype='<f4'))
    connectome["marker_color"] = to_color_strings(colors)
    connectome["markers_only"] = True
    return connectome



In [ ]:
# Choose the minimum cluster area (originally 50mm2)
min_area_threshold=50
# Choose the default way to assess the top cluster - can be mean, max or skew
clusters_by_mean_max="skew"
# Choose how many thresholds to test to pick optimal threshold - originally 21
thresholds=np.linspace(0,1,21)

In [ ]:
#Path to training subjects directory
pat_subjects_dir='/path/to/train_dir'
#Path to control subjects directory
cont_subjects_dir='/path/to/control_dir'



In [ ]:
# Creates list of training subjects ids - looks for patient ids beginning "FCD"
subject_ids=glob.glob(os.path.join(pat_subjects_dir,'FCD*'))
# List of patients to remove from subject IDs
Remove=['FCD_patient_1', 'FCD_patient_2']
patient_ids=[]
for subject in subject_ids:
    split_sub=subject.split('/')[-1]
    if split_sub not in Remove:
        patient_ids.append(split_sub)
        
#Save the list of training IDs        
np.savetxt('patient_ids_sEEG.txt',np.array(patient_ids), fmt = '%s')

In [ ]:
#Creates list of control ids - looks for IDs beginning C_
subject_ids=glob.glob(os.path.join(cont_subjects_dir,'C_*'))
control_ids=[]
for subject in subject_ids:
    split_sub=subject.split('/')[-1]
    if split_sub not in Remove:
        control_ids.append(split_sub)
#Saves the list of control IDs as a .txt file
np.savetxt('control_ids_sEEG.txt',np.array(control_ids), fmt = '%s')

In [ ]:
#preload fsaverage data (template data)
overlap=np.zeros_like(thresholds)
surf=os.path.join(pat_subjects_dir,'fsaverage_sym','surf','lh.inflated')
surface=mt.load_mesh_geometry(surf)
areas=os.path.join(pat_subjects_dir,'fsaverage_sym','surf','lh.area')
area=mt.load_mesh_data(areas)
neighbours=get_neighbours_from_tris(surface['faces'])



In [ ]:
#Create empty variables
# This variable will list whether there is an overlap between the cluster 
#and lesion label (i.e. correctly detected by classifer)
any_overlap=np.zeros(len(thresholds))
#Lists whether there is an overlap between the first (top) cluster 
#and lesion label (i.e. correctly detected by classifer)
first_cluster_overlap=np.zeros(len(thresholds))
#Lists number of predicted clusters
number_of_clusters=np.zeros(len(thresholds))

#Hemispheres
hemis=['rh','lh']
# For every patient
for subject_id in patient_ids:
    print(subject_id)
    # For every threshold
    for k, threshold in enumerate(thresholds):
        clusters_stacked=[]
        lesions_stacked=[]
        predictions_stacked=[]
        # for each hemisphere
        for hemi in hemis:
            # Change path and overlay to neural network output
            test_nn_output=os.path.join(pat_subjects_dir,subject_id,'xhemi','classifier',hemi+'.'+subject_id+'.classifier_output.mgh')
            
            # Change path and file name to lesion label
            mask=os.path.join(pat_subjects_dir,subject_id,'xhemi','surf',hemi+'.lesion_on_lh.mgh')

            prediction=io.load_mgh(test_nn_output)
            #if no lesion mask set all to zero
            try:
                lesion_mask=io.load_mgh(mask)
            except IOError:
                lesion_mask=np.zeros_like(prediction)
            #clusters above threshold
            clusters=cluster_above_threshold(prediction, np.array(neighbours), threshold=threshold)
            #removes clusters below minimum area (e.g. 50mm)
            clusters_filtered=filter_cluster(clusters,area,min_area_threshold)
            #within a subject creates a variable containing all clusters across hemispheres
            clusters_stacked.extend(clusters_filtered)
            #creates variable with lesion across both hemispheres
            lesions_stacked.extend(lesion_mask)
            #variable of neural network output across both hemis
            predictions_stacked.extend(prediction)
            #turns previous variables (lists) into arrays
        predictions_stacked=np.array(predictions_stacked)
        lesions_stacked=np.array(lesions_stacked)
        clusters_stacked=np.array(clusters_stacked)
        #ranks clusters
        clusters_ranked=rank_clusters(clusters_stacked,predictions_stacked,ranking="skew")
        #finds number of clusters
        number_of_clusters[k]=np.max(clusters_ranked)
        #finds if any clusters overlap with lesion label (ie, lesion detected by classifier)
        any_overlap[k]+=check_any_overlap_with_lesion_mask(clusters_ranked,lesions_stacked)
        # finds if first cluster overlaps with lesion label
        first_cluster_overlap[k]+=check_cluster_overlap_with_lesion_mask(clusters_ranked,lesions_stacked,cluster_index=1)

In [ ]:
#Gets specificity at every threshold
cont_any_lesions=np.zeros(len(thresholds))

hemis=['rh','lh']
# For each control participant
for subject_id in control_ids:
    print(subject_id)
    for k, threshold in enumerate(thresholds):
        clusters_stacked=[]
        predictions_stacked=[]
        for hemi in hemis:
            #Change path and file name to control classifier output
            test_nn_output=os.path.join(cont_subjects_dir,subject_id,'xhemi','classifier',hemi+'.'+subject_id+'.classifier_output.mgh')
            prediction=io.load_mgh(test_nn_output)
            #Clusters classifier output above the threshold
            clusters=cluster_above_threshold(prediction, np.array(neighbours), threshold=threshold)
            #Removes clusters below the threshold
            clusters_filtered=filter_cluster(clusters,area,min_area_threshold)
            # Creates list of clusters across both hemis
            clusters_stacked.extend(clusters_filtered)
            # Creates list of neural network output across both hemis
            predictions_stacked.extend(prediction)
            #Converts lists to arrays
        predictions_stacked=np.array(predictions_stacked)
        clusters_stacked=np.array(clusters_stacked)
        #ranks clusters
        clusters_ranked=rank_clusters(clusters_stacked,predictions_stacked,ranking="max")
        #Checks if each control has any detected areas
        cont_any_lesions[k]+=(clusters_ranked>0).any()

In [ ]:
#Sensitivity is a variable containing the sensitivity of the classifier at all tested thresholds
sensitivity=any_overlap/len(patient_ids)
#Specificity is a variable containing the specificity of the classifier at all tested thresholds
specificity=1-cont_any_lesions/len(control_ids)

In [ ]:
#Plot sensitivity and specificity at different thresholds
# x axis = thresholds
# y axis = sensitivity + specificity
%matplotlib inline
plt.plot(thresholds,sensitivity+(specificity))

# calculates optimal threshold - peak of the youden index
optimal_threshold=get_optimal_threshold(sensitivity,specificity,thresholds,resolution=10)
print(optimal_threshold)

In [ ]:
#For the optimal theshold, calculate the number of clusters in each patient and whether lesion detected
import nibabel as nb
#Sets threshold as optimal threshold
threshold = optimal_threshold
hemis=['rh','lh']
pat_number_of_clusters=np.zeros(len(patient_ids))
pat_any_overlap=np.zeros(len(patient_ids))
pat_first_cluster_overlap=np.zeros(len(patient_ids))

# In every subject
for k,subject_id in enumerate(patient_ids):
    print(subject_id)
    clusters_stacked=[]
    lesions_stacked=[]
    predictions_stacked=[]
    # In each hemisphere
    for h,hemi in enumerate(hemis):
        #Path to classifier output in xhemi space(range of values 0 to 1 where values closer to 1 are more likely to be lesional)
        test_nn_output=os.path.join(pat_subjects_dir,subject_id,'xhemi','classifier',hemi+'.'+subject_id+'.classifier_output.mgh')
        #Path to lesion mask in xhemi space (this is the lesion mask created manually in native space and registered to xhemi)
        mask=os.path.join(pat_subjects_dir,subject_id,'xhemi','surf',hemi+'.lesion_on_lh.mgh')
        # This loads the classifier output
        prediction=io.load_mgh(test_nn_output)
        #This checks if there is a lesion mask for the hemisphere - if no lesion mask sets all values of variable prediction to zero
        try:
            lesion_mask=io.load_mgh(mask)
        except IOError:
             lesion_mask=np.zeros_like(prediction)
        #This filters the classifier output above a threshold and then clusters the values above the threshold
        clusters=cluster_above_threshold(prediction, np.array(neighbours), threshold=threshold)
        #This filters out any clusters below the min_area_threshold (originally set to be 50mm2)
        clusters_filtered=filter_cluster(clusters,area,min_area_threshold)
        #This ranks the clusters. Clusters can be ranked according to mean, max, skew etc. 
        clusters_ordered=rank_clusters(clusters_filtered,prediction,ranking="skew")
        # This joins (groups) clusters that are within a number of vertices (n_steps) together
        grouped=clump_clusters(clusters_ordered,np.array(neighbours),n_steps=5)
        # This fills in holes in clusters e.g. if all neighbours are in the cluster, vertex will become part of cluster
        defragged=defrag_surface(grouped, np.array(neighbours),steps=5)
        defragged[defragged>0]+=h*1000

        #This sets the final clusters into a variable called clusters_stacked which includes the data from both hemispheres
        clusters_stacked.extend(defragged)
        #This creates a variable called lesions_stacked that contains the lesion mask across both hemispheres
        lesions_stacked.extend(lesion_mask)
        #This creates a variable predictions_stacked that contains the original classifier output for both hemispheres
        predictions_stacked.extend(prediction)
    #This changes the variables predictions_stacked, lesions_stacked, clusters_stacked from a list into an array so that numpy commands can be used
    predictions_stacked=np.array(predictions_stacked)
    lesions_stacked=np.array(lesions_stacked)
    clusters_stacked=np.array(clusters_stacked)
    #This reranks the clusters across both hemispheres. Clusters can be ranked according to mean, max, skew, etc.
    clusters_ranked=rank_clusters(clusters_stacked,predictions_stacked,ranking="skew")
    #This calculates the total number of clusters for the patient
    pat_number_of_clusters[k]=np.max(clusters_ranked)
    print(pat_number_of_clusters[k])
    #This calculates whether any of the clusters overlap with the lesion label (sensitivity)
    pat_any_overlap[k]=check_any_overlap_with_lesion_mask(clusters_ranked,lesions_stacked)
    print(pat_any_overlap[k])
    #This calculates whether the top cluster overlaps with the lesion label 
    pat_first_cluster_overlap[k]=return_cluster_overlap_with_lesion_mask(clusters_ranked,lesions_stacked)[0]
    #This prints the cluster number that overlaps with the lesion label
    print(pat_first_cluster_overlap)

In [ ]:
# Prints sensitivity, percentage of patients where the top cluster overlaps the lesion label
# and prints average number of clusters per training patient at optimal threshold
Sensitivity = np.mean(pat_any_overlap>0)
print(Sensitivity)
Percentage_first_cluster = np.mean(pat_first_cluster_overlap==1)
print(Percentage_first_cluster)
Mean_no_clusters =np.mean(pat_number_of_clusters)
print(Mean_no_clusters)

In [ ]:
# FIND OUT SPECICITY IN CONTROLS
#Create variable called control number of clusters
cont_number_of_clusters=np.zeros(len(control_ids))

# In every control
for k,subject_id in enumerate(control_ids):
    print(subject_id)
    clusters_stacked=[]
    predictions_stacked=[]
    # In each hemisphere
    for hemi in hemis:
        #Path to classifier output in xhemi space(range of values 0 to 1 where values closer to 1 are more likely to be lesional)
        test_nn_output=os.path.join(cont_subjects_dir,subject_id,'xhemi','classifier',hemi+'.'+subject_id+'.classifier_output.mgh')
        # This loads the classifier output
        prediction=io.load_mgh(test_nn_output)
        #This filters the classifier output above a threshold and then clusters the values above the threshold
        clusters=cluster_above_threshold(prediction, np.array(neighbours), threshold=threshold)
        #This filters out any clusters below the min_area_threshold (originally set to be 50mm2)
        clusters_filtered=filter_cluster(clusters,area,min_area_threshold)
        #This ranks the clusters. Clusters can be ranked according to mean, max, skew etc. 
        clusters_ordered=rank_clusters(clusters_filtered,prediction,ranking="skew")
        # This joins (groups) clusters that are within a number of vertices (n_steps) together
        grouped=clump_clusters(clusters_ordered,np.array(neighbours),n_steps=5)
        # This fills in holes in clusters e.g. if all neighbours are in the cluster, vertex will become part of cluster
        defragged=defrag_surface(grouped, np.array(neighbours),steps=5)
        defragged[defragged>0]+=k*1000

        #This sets the final clusters into a variable called clusters_stacked which includes the data from both hemispheres
        clusters_stacked.extend(defragged)
        #This creates a variable predictions_stacked that contains the original classifier output for both hemispheres
        predictions_stacked.extend(prediction)
    #This changes the variables predictions_stacked, lesions_stacked, clusters_stacked from a list into an array so that numpy commands can be used
    predictions_stacked=np.array(predictions_stacked)
    clusters_stacked=np.array(clusters_stacked)
    #This reranks the clusters across both hemispheres. Clusters can be ranked according to mean, max, skew, etc.
    clusters_ranked=rank_clusters(clusters_stacked,predictions_stacked,ranking="skew")
    #This calculates the total number of clusters for the patient
    cont_number_of_clusters[k]=np.max(clusters_ranked)
    print(cont_number_of_clusters[k])

#Prints the number of clusters in each control participant
print(cont_number_of_clusters)
Mean_no_clusters_controls =np.mean(cont_number_of_clusters)
#Prints the average number of clusters per control
print(Mean_no_clusters_controls)

In [ ]:
import nibabel as nb
def process_predictions_to_ranked_clusters(seeg_subjects_dir,subject_id, neighbours, threshold, ranking="skew",steps=5):
    """Load in classifier output (range of values between 0 and 1 where values closer to 1 more likely to be lesional)
     Filter, cluster and rank the clusters across both hemispheres
     Save out ranked clusters as"""
    clusters_stacked=[]
    predictions_stacked=[]
    # In each hemisphere
    hemis=['rh','lh']
    for k, hemi in enumerate(hemis):
        #Path to classifier output in xhemi space(range of values 0 to 1 where values closer to 1 are more likely to be lesional)
        #test_nn_output=os.path.join(seeg_subjects_dir,subject_id,'xhemi','classifier',hemi+'.'+subject_id+'.classifier_output.mgh')
        test_nn_output=os.path.join(seeg_subjects_dir,subject_id,'xhemi','classifier',hemi+'.'+subject_id+'.classifier_ouput.mgh')
        # This loads the classifier output
        prediction=io.load_mgh(test_nn_output)
        #This filters the classifier output above a threshold and then clusters the values above the threshold
        clusters=cluster_above_threshold(prediction, np.array(neighbours), threshold=threshold)
        #This filters out any clusters below the min_area_threshold (originally set to be 50mm2)
        clusters_filtered=filter_cluster(clusters,area,min_area_threshold)
        #This ranks the clusters. Clusters can be ranked according to mean, max, skew etc. 
        clusters_ordered=rank_clusters(clusters_filtered,prediction,ranking=ranking)
        print(clusters_ordered)
        # This joins (groups) clusters that are within a number of vertices (n_steps) together
        grouped=clump_clusters(clusters_ordered,np.array(neighbours),n_steps=steps)
        # This fills in holes in clusters e.g. if all neighbours are in the cluster, vertex will become part of cluster

        defragged=defrag_surface(grouped, np.array(neighbours),steps=steps)
        defragged[defragged>0]+=k*1000

        #This sets the final clusters into a variable called clusters_stacked which includes the data from both hemispheres
        clusters_stacked.extend(defragged)
        #This creates a variable predictions_stacked that contains the original classifier output for both hemispheres
        predictions_stacked.extend(prediction)
    #This changes the variables predictions_stacked, lesions_stacked, clusters_stacked from a list into an array so that numpy commands can be used
    predictions_stacked=np.array(predictions_stacked)
    clusters_stacked=np.array(clusters_stacked)
    print(np.unique(clusters_stacked))
    #This reranks the clusters across both hemispheres. Clusters can be ranked according to mean, max, skew, etc.
    clusters_ranked=rank_clusters(clusters_stacked,predictions_stacked,ranking=ranking)
    clusters_ranked_rh=clusters_ranked[:len(clusters_ranked)/2]
    clusters_ranked_lh=clusters_ranked[len(clusters_ranked)/2:]
    
    
    print("right",np.unique(clusters_ranked_rh))
    print("left",np.unique(clusters_ranked_lh))
    print("both", np.unique(clusters_ranked))
    demo=nb.load(test_nn_output)
    # Saves an overlay for each hemi of the clusters after thresholding
    io.save_mgh(os.path.join(seeg_subjects_dir,subject_id,'xhemi','classifier','lh.'+subject_id+'.classifier_output_clusters_after_thresholding.mgh'),clusters_ranked_lh,demo)
    io.save_mgh(os.path.join(seeg_subjects_dir,subject_id,'xhemi','classifier','rh.'+subject_id+'.classifier_output_clusters_after_thresholding.mgh'),clusters_ranked_rh,demo)
    return 

In [ ]:
# Load in SEEG data, threshold, cluster, plot electrodes and plot clusters and save as html and finally evaluate proximity of clusters to ictal contacts

excel_path='/path/to/excel'
## Load in your excel file containing seeg ictal, interictal, spread fiducials
xls = pd.ExcelFile(os.path.join(excel_path,'sEEG_data.xlsx'))
## Change path to location of sEEG data 
meld_path = '/path/to/seeg'
## Change to location of freesurfer data for sEEG subjects
freesurfer_dir = '/path/to/Freesurfer/'
## List any patients you want to exclude
exclude=np.array([['FCD_patient_1'],['FCD_patient_2']])




In [ ]:
# Read in interictal, ictal, all fiducials and affected hemisphere from excel spreadsheet
Interictal = pd.read_excel( xls, 'Interictal' )
Ictal = pd.read_excel( xls, 'Ictal')
Fiducials = pd.read_excel( xls, 'Fiducials')
Hemi = pd.read_excel( xls, 'Hemi')
Included = pd.read_excel( xls, 'Included')



In [ ]:
#get subject_ids for seeg patients
subject_ids =Included.loc[:,'ID'].values

# Set cluster number to assess proximity of ictal fiducials from 
cluster_no=1

print(subject_ids)

In [ ]:
#preload fsaverage data
overlap=np.zeros_like(thresholds)
surf=os.path.join(pat_subjects_dir,'fsaverage_sym','surf','lh.inflated')
surface=mt.load_mesh_geometry(surf)
areas=os.path.join(pat_subjects_dir,'fsaverage_sym','surf','lh.area')
area=mt.load_mesh_data(areas)
neighbours=get_neighbours_from_tris(surface['faces'])

In [ ]:
# Load in classifier output (range of values between 0 and 1 where values closer to 1 more likely to be lesional)
# Filter, cluster and rank the clusters across both hemispheres
# Save out ranked clusters as "classifier_output_clusters_after_thresholding.mgh"

for sub in subject_ids:
    if sub in exclude:
        continue
    print(sub)
    process_predictions_to_ranked_clusters(freesurfer_dir,sub, neighbours, optimal_threshold)



In [ ]:
# Use cell below to plot 
label_colors=np.array(['#aaaaaa', '#FF5733','#FFC300'])

for sub in subject_ids:
    #if os.path.isdir(os.path.join(meld_path,sub)):
    if os.path.isdir(os.path.join(meld_path,sub+'_recon')):
        print(sub)
        if sub in exclude:
            continue
        #Gets affected hemisphere of patient from excel    
        hemi = np.array(Hemi['Hemi'][Hemi['ID'] == sub ])[0]
        #Gets list of fiducials (electrodes) for patient from excel
        fiducials = get_fiducials(sub)
        #Prints list of electrodes implanted
        print(fiducials)
        # Gets list of ictal contacts
        ictals = get_electrodes(Ictal, sub)
        #Gets list of interictal contacts
        interictals = get_electrodes(Interictal, sub)
        colours=[]
        electrode_coords=[]
        for f in fiducials:
            #Loads .fcsv file of electrode
            electrode = np.genfromtxt(os.path.join(meld_path, sub, 'Slicer',f+'.fcsv'),delimiter=',', usecols = (0,1,2,3,4))
            #check if only one electrode
            if np.ndim(electrode)==1:
                length_seeg = 1
                #finds electrode contact-co-ordinates
                electrode_coords=electrode[1:4]
            else:
                length_seeg=len(electrode[:,1:4])
                if len(electrode_coords)==0:
                    electrode_coords=electrode[:,1:4]
                else:
                    #Finds electrode contact co-ords
                    electrode_coords=np.vstack((electrode_coords,electrode[:,1:4]))
            #Codes ictal contacts red, interictals yellow and other contacts black
            for k in range(length_seeg):
                contact_name = f + str(k+1)
                if contact_name in ictals:
                    contact_value = 'red'
                    print(contact_name)
                elif contact_name in interictals:
                    contact_value = 'yellow'
                   # print(contact_name)
                else :
                    contact_value = 'black'
                colours.append(contact_value)

## The next part of the script creates an interactive plot and saves as an .html of the electrodes and the classifier output
#It uses nilearn plotting.
        connectome_info = _get_markers(electrode_coords, colours)
        connectome_info["marker_size"] = 10 #marker_size
        plot_info = {"connectome": connectome_info}

        hemis=['right', 'left']
        hemifs=['rh','lh']
        replacement='["left", "right","labelleft","labelright"]'



        for h,hemi in enumerate(hemis):
        #plot gray translucent surface
            pialname=os.path.join(meld_path,sub+'_recon','surf/'+hemifs[h]+'.pial')
            whitename=os.path.join(meld_path,sub+'_recon','surf/'+hemifs[h]+'.white')


            lesion_name=os.path.join(meld_path,sub+'_recon','surf/'+hemifs[h]+'.'+sub+'.classifier_output_clusters_after_thresholding.mgh')
            lesion_surf=os.path.join(meld_path,sub+'_recon','surf',hemifs[h]+'.lesions.pial')
            plot_info['pial_'+hemi]=mesh_to_plotly(pialname)    
            neighbours=get_neighbours_from_tris(io_mesh.load_mesh_geometry(pialname)['faces'])
            surf_map=tidy_integer_labels(np.array(io.import_mgh(lesion_name)),neighbours)

            #set lesions above 1 to all same colour (2)
            surf_map[surf_map>1]=2
            plot_info['vertexcolor_'+hemi] = list(label_colors[surf_map])
            plot_info['opacity_'+hemi] = 0.1

            #plot opaque lesion labels
            label=np.where(surf_map!=0)[0]  
            label2object(lesion_surf,pialname,whitename,label)
            # try because if no lesions on one side, no mesh is created and html fails
            #so remove labelleft or labelright to plot
            try:
                plot_info['pial_label'+hemi] = mesh_to_plotly(lesion_surf)
                plot_info['vertexcolor_label'+hemi] = list(np.hstack((label_colors[surf_map],label_colors[surf_map])))
                plot_info['opacity_label'+hemi] = 0.4
            except ValueError:
                if hemi=='right':
                    replacement='["left", "right","labelleft"]'
                elif hemi=='left':
                    replacement='["left", "right","labelright"]'

        as_json = json.dumps(plot_info)
        as_html = get_html_template('connectome_plot_template.html')
        as_html= as_html.replace('["left", "right"]',replacement)

        as_html= as_html.replace('info["color"] = "#aaaaaa";','info["vertexcolor"] = connectomeInfo["vertexcolor_" + hemisphere];')
        as_html= as_html.replace('info["opacity"] = getOpacity();','info["opacity"] = connectomeInfo["opacity_" + hemisphere];')

        as_html=as_html.replace(
            'INSERT_CONNECTOME_JSON_HERE', as_json)
        as_html = add_js_lib(as_html, embed_js=True)

        class ConnectomeView(HTMLDocument):
            pass
        ConnectomeView(as_html).save_as_html(os.path.join(meld_path,sub,sub+'.html'))
        
## The next part of the script plots a raincloud plot of distance of the ictal, interictal 
#and other contacts from a cluster. This is plotted for clusters 1 to 5.

        for counter in range(5):
            #not interested in number 0
            cluster_no=counter+1
            lesion_coords=[[0]]
            for h,hemi in enumerate(hemis):
                pialname=os.path.join(meld_path,sub+'_recon','surf/'+hemifs[h]+'.pial')
                whitename=os.path.join(meld_path,sub+'_recon','surf/'+hemifs[h]+'.white')
                #lesion_name=os.path.join(meld_path,sub+'_recon','surf/'+hemifs[h]+'.'+sub+'.classifier_output_clusters_after_thresholding.mgh')
                lesion_name=os.path.join(meld_path,sub+'_recon','surf/'+hemifs[h]+'.'+sub+'.classifier_output_clusters_after_thresholding.mgh')
                dum=lesion_coordinates(pialname,whitename,lesion_name,cluster_no)
                if len(dum) >0:
                    lesion_coords=dum
            if lesion_coords[0][0]==0:
                print("no cluster found for this cluster number, exiting loop")
                break
            distances=np.zeros(np.shape(colours))
            for k,coord in enumerate(electrode_coords):
                closest_coord, distances[k]=closest_node(coord,lesion_coords)
            classification=[]
            poss_colours=['black','yellow','red']
            for node in colours:
                classification.append(poss_colours.index(node))
            f, ax = plt.subplots(figsize=(12, 11))
            colors=['black','orange','red']
            plt.rcParams.update({'font.size': 30})
            threshold=10
            border=3
            a=plt.plot([-2,3],[threshold,threshold],'r')
            a=plt.fill_between([-2,3],[threshold-border,threshold-border],[threshold+border,threshold+border],facecolor='#ffdab9',alpha=0.2)

            dy = "Distance from predicted lesion (mm)"; dx = "Electrode classification"; ort = "v"
            ax=pt.half_violinplot(classification,distances, palette=sns.color_palette(colors), bw=.2,  linewidth=1,cut=0.,\
                               scale="area", width=1, inner=None,orient="v")
            ax=sns.stripplot(classification,distances, palette=sns.color_palette(colors), edgecolor="white",size=7,orient="v",\
                             jitter=1,zorder=0)
            ax=sns.boxplot(classification,distances, color="black",orient="v",width=.15,zorder=10,\
                          showcaps=True,boxprops={'facecolor':'none', "zorder":10},\
                           showfliers=True,whiskerprops={'linewidth':2, "zorder":10},saturation=1)

            a=plt.ylabel(dy)
            a=plt.xlabel(dx)

            a=plt.xticks([0,1,2],['Nondescript','Interictal','Ictal'])
            a=plt.savefig(os.path.join(meld_path,sub,sub+'_raincloud_'+str(cluster_no)+'.png'))